In [4]:
"""
UDI-DI (Unique Device Identifier - Device Identifier) 클린징 모듈

4가지 UDI 발행 기관:
1. GS1: GTIN 형식 (8, 12, 13, 14자리 숫자)
2. HIBCC: + 기호로 시작
3. ICCBBA: = 기호로 시작  
4. NDC/NHRIC: 특정 형식

참고: https://health.ec.europa.eu/medical-devices-topics-interest/unique-device-identifier-udi_en
"""

import re
from typing import Tuple, Optional


class UDICleaner:
    """UDI-DI 클린징 클래스"""
    
    # 삭제할 이상 케이스 패턴
    INVALID_PATTERNS = [
        r'^0+$',                    # 모두 0 (00000000 등)
        r'^UNKNOWN$',               # UNKNOWN
        r'^UNK$',                   # UNK
        r'^N/A$',                   # N/A
        r'^NA$',                    # NA
        r'^NULL$',                  # NULL
        r'^NONE$',                  # NONE
        r'^NOT AVAILABLE$',         # NOT AVAILABLE
        r'^UNAVAILABLE$',           # UNAVAILABLE
        r'^\s*$',                   # 빈 문자열
        r'^N$',                     # 단일 문자 N
        r'.*UNKNOWN.*',             # UNKNOWN 포함
        r'.*\(01\).*NA.*',          # (01)과 NA 포함
        r'.*\$\$.*',                # $$ 포함 (이상 문자)
        r'^0{8,}$',                 # 8개 이상의 연속된 0
    ]
    
    # GS1 Application Identifier 패턴
    GS1_AI_PATTERN = r'\(01\)'
    
    def __init__(self):
        self.stats = {
            'total': 0,
            'invalid_removed': 0,
            'gs1_cleaned': 0,
            'hibcc_cleaned': 0,
            'iccbba_cleaned': 0,
            'model_number': 0,
            'kept_as_is': 0,
            'failed': 0
        }
    
    def clean(self, udi_di: str) -> Tuple[Optional[str], str]:
        """
        UDI-DI를 클린징
        
        Args:
            udi_di: 원본 UDI-DI 문자열
            
        Returns:
            (cleaned_udi, status): 클린징된 UDI-DI와 상태
                status: 'VALID', 'INVALID', 'MODEL_NUMBER', 'CLEANED'
        """
        self.stats['total'] += 1
        
        # 전처리: 따옴표, 공백 제거
        udi = str(udi_di).strip().strip("'\"")
        
        # 1. 이상 케이스 체크 (삭제 대상)
        if self._is_invalid(udi):
            self.stats['invalid_removed'] += 1
            return None, 'INVALID'
        
        # 2. GS1 형식 클린징
        if self._is_gs1_format(udi):
            cleaned = self._clean_gs1(udi)
            if cleaned:
                self.stats['gs1_cleaned'] += 1
                return cleaned, 'CLEANED'
            else:
                # GS1 형식이지만 클린징 실패 -> INVALID
                self.stats['invalid_removed'] += 1
                return None, 'INVALID'
        
        # 3. HIBCC 형식 클린징
        if self._is_hibcc_format(udi):
            cleaned = self._clean_hibcc(udi)
            if cleaned:
                self.stats['hibcc_cleaned'] += 1
                return cleaned, 'CLEANED'
            else:
                # HIBCC 형식이지만 클린징 실패 -> INVALID
                self.stats['invalid_removed'] += 1
                return None, 'INVALID'
        
        # 4. ICCBBA 형식 클린징
        if self._is_iccbba_format(udi):
            cleaned = self._clean_iccbba(udi)
            if cleaned:
                self.stats['iccbba_cleaned'] += 1
                return cleaned, 'CLEANED'
            else:
                # ICCBBA 형식이지만 클린징 실패 -> INVALID
                self.stats['invalid_removed'] += 1
                return None, 'INVALID'
        
        # 5. 모델번호로 추정되는 경우
        if self._is_model_number(udi):
            self.stats['model_number'] += 1
            return f"MODEL:{udi}", 'MODEL_NUMBER'
        
        # 6. 길이가 너무 짧거나 긴 경우 (이상 케이스)
        if len(udi) < 6 or len(udi) > 30:
            self.stats['invalid_removed'] += 1
            return None, 'INVALID'
        
        # 7. 그 외는 원본 유지 (추가 검토 필요)
        self.stats['kept_as_is'] += 1
        return udi, 'VALID'
    
    def _is_invalid(self, udi: str) -> bool:
        """이상 케이스 체크"""
        if not udi or len(udi) == 0:
            return True
        
        # 정규표현식 패턴 매칭
        for pattern in self.INVALID_PATTERNS:
            if re.match(pattern, udi, re.IGNORECASE):
                return True
        
        # 추가 검증: 이상한 문자 포함 체크
        # 허용 문자: 숫자, 알파벳, +, =, -, /, (, ), :
        if re.search(r'[^0-9A-Za-z+=\-/():.]', udi):
            return True
        
        # 00000000 같은 무의미한 패딩 체크 (모두 같은 숫자)
        if len(udi) >= 8 and len(set(udi)) == 1:
            return True
        
        return False
    
    def _is_gs1_format(self, udi: str) -> bool:
        """GS1 형식 여부 확인"""
        # 숫자로만 구성 or GS1 AI 포함
        return (re.match(r'^\d+$', udi) or 
                re.search(self.GS1_AI_PATTERN, udi) is not None or
                udi.startswith('(01)'))
    
    def _is_hibcc_format(self, udi: str) -> bool:
        """HIBCC 형식 여부 확인 (+ 기호로 시작 or 숫자+기호로 시작)"""
        # 정상적인 HIBCC
        if udi.startswith('+'):
            return True
        # 앞에 숫자가 붙은 HIBCC (0+, 00+ 등)
        if re.match(r'^\d+\+', udi):
            return True
        return False
    
    def _is_iccbba_format(self, udi: str) -> bool:
        """ICCBBA 형식 여부 확인 (= 기호로 시작)"""
        return udi.startswith('=')
    
    def _is_model_number(self, udi: str) -> bool:
        """
        모델번호 여부 확인
        - M으로 시작하고 문자와 숫자 혼합
        - D로 시작하고 문자 포함 (일부 의료기기 모델번호)
        """
        if not udi:
            return False
        
        # M으로 시작하고 길이가 6자 이상, 문자와 숫자 혼합
        if udi.startswith('M') and len(udi) >= 6:
            has_letter = bool(re.search(r'[A-Za-z]', udi))
            has_digit = bool(re.search(r'\d', udi))
            return has_letter and has_digit
        
        # D로 시작하고 문자 포함 (숫자만 있는 경우 제외)
        if udi.startswith('D') and len(udi) >= 6:
            has_letter = bool(re.search(r'[A-Za-z]{2,}', udi))  # 2개 이상의 문자
            return has_letter
        
        return False
    
    def _clean_gs1(self, udi: str) -> Optional[str]:
        """
        GS1 형식 클린징
        - GS1 GTIN: 8, 12, 13, 14자리 숫자
        - Application Identifier 제거
        - 앞/뒤 0 패딩 조정
        """
        # GS1 AI가 포함된 경우 전체에서 숫자만 추출
        # 예: 00(01)(21)5121 -> 00005121
        udi = re.sub(r'[^\d]', '', udi)
        
        if not udi or len(udi) == 0:
            return None
        
        # 너무 짧은 경우 (8자리 미만)
        if len(udi) < 8:
            # 8자리로 패딩 시도
            udi = udi.zfill(8)
        
        # GS1 GTIN 표준 길이로 정규화
        # 8, 12, 13, 14자리가 표준
        length = len(udi)
        
        # 14자리 미만이면서 10자리 이상인 경우 14자리로 패딩
        if 10 <= length < 14:
            udi = udi.zfill(14)
        
        # 14자리 초과하는 경우 (18자리 등)
        # 앞쪽 0을 제거하여 14자리로 맞춤
        elif length > 14:
            # 앞쪽 0 제거 후 14자리 유지
            udi = udi.lstrip('0')
            if len(udi) > 14:
                # 여전히 길면 앞에서 14자리만
                udi = udi[:14]
            elif len(udi) < 14:
                # 짧아지면 다시 패딩
                udi = udi.zfill(14)
        
        # 최종 검증: 8, 12, 13, 14자리 중 하나여야 함
        if len(udi) not in [8, 12, 13, 14]:
            return None
        
        # 모두 같은 숫자인지 체크 (00000000 등)
        if len(set(udi)) == 1:
            return None
        
        return udi
    
    def _clean_hibcc(self, udi: str) -> Optional[str]:
        """
        HIBCC 형식 클린징
        - Primary/Secondary 분석 결과: + 기호를 제거해야 매핑됨
        - 앞쪽에 숫자가 붙은 경우 (0+, 00+) 제거
        """
        # 앞쪽 숫자 제거 (0+, 00+ 등)
        udi = re.sub(r'^0+\+', '+', udi)
        
        if not udi.startswith('+'):
            return None
        
        # + 기호 다음의 내용 추출
        cleaned = udi[1:].strip()
        
        # HIBCC 형식 검증: 알파벳+숫자 조합
        if len(cleaned) < 4:
            return None
        
        # 옵션 1: + 제거 (Primary/Secondary 데이터셋 매핑을 위해)
        # 실제 매핑 시에는 + 없이 사용
        return cleaned
        
        # 옵션 2: + 유지 (표준 형식)
        # return udi
    
    def _clean_iccbba(self, udi: str) -> Optional[str]:
        """
        ICCBBA 형식 클린징
        - = 기호로 시작
        """
        if not udi.startswith('='):
            return None
        
        # = 기호 다음의 내용 추출
        cleaned = udi[1:].strip()
        
        if len(cleaned) < 4:
            return None
        
        # = 제거 옵션 (HIBCC와 동일한 로직)
        return cleaned
    
    def print_stats(self):
        """통계 출력"""
        print("\n" + "="*60)
        print("UDI-DI 클린징 통계")
        print("="*60)
        print(f"전체 레코드:           {self.stats['total']:,}")
        print(f"이상 케이스 삭제:      {self.stats['invalid_removed']:,}")
        print(f"GS1 클린징:            {self.stats['gs1_cleaned']:,}")
        print(f"HIBCC 클린징:          {self.stats['hibcc_cleaned']:,}")
        print(f"ICCBBA 클린징:         {self.stats['iccbba_cleaned']:,}")
        print(f"모델번호로 변환:       {self.stats['model_number']:,}")
        print(f"원본 유지:             {self.stats['kept_as_is']:,}")
        print(f"처리 실패:             {self.stats['failed']:,}")
        print("="*60)
        
        # 클린징 비율
        if self.stats['total'] > 0:
            cleaned = (self.stats['gs1_cleaned'] + self.stats['hibcc_cleaned'] + 
                      self.stats['iccbba_cleaned'] + self.stats['model_number'])
            print(f"\n클린징 성공률: {cleaned / self.stats['total'] * 100:.2f}%")
            print(f"삭제율: {self.stats['invalid_removed'] / self.stats['total'] * 100:.2f}%")


def clean_udi_file(input_file: str, output_file: str, verbose: bool = True):
    """
    UDI-DI 파일 클린징
    
    Args:
        input_file: 입력 파일 경로
        output_file: 출력 파일 경로
        verbose: 상세 출력 여부
    """
    cleaner = UDICleaner()
    
    # 파일 읽기
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()
        udi_list = eval(content)  # Python set/list 형식
    
    if verbose:
        print(f"\n입력 파일: {input_file}")
        print(f"총 {len(udi_list):,}개의 UDI-DI 로드됨")
    
    # 클린징
    cleaned_udis = []
    invalid_udis = []
    model_numbers = []
    
    for udi in udi_list:
        cleaned, status = cleaner.clean(udi)
        
        if status == 'INVALID':
            invalid_udis.append(udi)
        elif status == 'MODEL_NUMBER':
            model_numbers.append((udi, cleaned))
        elif cleaned:
            cleaned_udis.append(cleaned)
    
    # 결과 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("{\n")
        for i, udi in enumerate(sorted(set(cleaned_udis))):
            if i < len(set(cleaned_udis)) - 1:
                f.write(f"    '{udi}',\n")
            else:
                f.write(f"    '{udi}'\n")
        f.write("}")
    
    # 모델번호 별도 저장
    if model_numbers:
        model_file = output_file.replace('.txt', '_models.txt')
        with open(model_file, 'w', encoding='utf-8') as f:
            f.write("# 원본 -> 모델번호\n")
            for original, model in model_numbers:
                f.write(f"{original} -> {model}\n")
    
    # 이상 케이스 별도 저장
    if invalid_udis:
        invalid_file = output_file.replace('.txt', '_invalid.txt')
        with open(invalid_file, 'w', encoding='utf-8') as f:
            f.write("# 삭제된 이상 케이스\n")
            for udi in invalid_udis:
                f.write(f"{udi}\n")
    
    if verbose:
        cleaner.print_stats()
        print(f"\n출력 파일: {output_file}")
        print(f"클린징된 UDI-DI: {len(set(cleaned_udis)):,}개")
        if model_numbers:
            print(f"모델번호: {len(model_numbers):,}개 ({model_file})")
        if invalid_udis:
            print(f"삭제된 이상 케이스: {len(invalid_udis):,}개 ({invalid_file})")


if __name__ == "__main__":
    import sys
    
    # 사용 예시
    print("UDI-DI 클린징 프로그램")
    print("="*60)
    
    # notfound.txt 클린징
    input_file = './notfound.txt'
    output_file = './notfound_cleaned.txt'
    
    print(f"\n클린징 시작: {input_file}")
    clean_udi_file(input_file, output_file, verbose=True)
    
    print("\n✓ 클린징 완료!")

UDI-DI 클린징 프로그램

클린징 시작: ./notfound.txt

입력 파일: ./notfound.txt
총 30,804개의 UDI-DI 로드됨

UDI-DI 클린징 통계
전체 레코드:           30,804
이상 케이스 삭제:      27
GS1 클린징:            30,378
HIBCC 클린징:          26
ICCBBA 클린징:         0
모델번호로 변환:       145
원본 유지:             228
처리 실패:             0

클린징 성공률: 99.17%
삭제율: 0.09%

출력 파일: ./notfound_cleaned.txt
클린징된 UDI-DI: 29,236개
모델번호: 145개 (./notfound_cleaned_models.txt)
삭제된 이상 케이스: 27개 (./notfound_cleaned_invalid.txt)

✓ 클린징 완료!


In [7]:
"""
개선된 UDI-DI 이상 패턴 처리
- 유연한 패턴 매칭
- 패턴 제거 vs 완전 삭제 구분
"""

import re
from typing import Tuple, Optional


class ImprovedInvalidPatterns:
    """
    개선된 이상 패턴 처리 클래스
    
    각 패턴은 (pattern, action, description) 튜플로 구성:
    - pattern: 정규표현식 패턴
    - action: 'DELETE' (완전 삭제) 또는 'REMOVE' (패턴만 제거)
    - description: 패턴 설명
    """
    
    INVALID_PATTERNS = [
        # ========== DELETE: 완전히 삭제 (NULL 처리) ==========
        # 의미 없는 데이터 - 복구 불가능
        
        (r'^0+$', 'DELETE', '모두 0'),
        (r'^0{8,}$', 'DELETE', '8개 이상 연속된 0'),
        (r'^(\d)\1+$', 'DELETE', '같은 숫자만 반복 (11111, 22222 등)'),
        
        (r'^\s*$', 'DELETE', '빈 문자열'),
        (r'^N$', 'DELETE', '단일 문자 N'),
        (r'^X+$', 'DELETE', 'X만 반복'),
        
        # 명시적 결측값 표현
        (r'^UNKNOWN$', 'DELETE', 'UNKNOWN'),
        (r'^UNK(NOWN)?$', 'DELETE', 'UNK/UNKNOWN'),
        (r'^N/?A$', 'DELETE', 'N/A 또는 NA'),
        (r'^NULL$', 'DELETE', 'NULL'),
        (r'^NONE$', 'DELETE', 'NONE'),
        (r'^NOT[\s_]?AVAILABLE$', 'DELETE', 'NOT AVAILABLE'),
        (r'^UNAVAILABLE$', 'DELETE', 'UNAVAILABLE'),
        (r'^MISSING$', 'DELETE', 'MISSING'),
        (r'^NO[\s_]?DATA$', 'DELETE', 'NO DATA'),
        (r'^EMPTY$', 'DELETE', 'EMPTY'),
        
        # 테스트/더미 데이터
        (r'^TEST\d*$', 'DELETE', 'TEST 데이터'),
        (r'^DUMMY$', 'DELETE', 'DUMMY 데이터'),
        (r'^SAMPLE$', 'DELETE', 'SAMPLE 데이터'),
        
        # 너무 짧거나 긴 경우 (길이 기반)
        (r'^.{1,5}$', 'DELETE', '너무 짧음 (5자 이하)'),
        (r'^.{31,}$', 'DELETE', '너무 김 (31자 이상)'),
        
        # ========== REMOVE: 패턴만 제거하고 나머지 유지 ==========
        # 수정 가능한 데이터 - 패턴 제거 후 재사용
        
        # 불필요한 공백 및 구분자
        (r'\s+', 'REMOVE', '공백'),
        
        # GS1 Application Identifiers - 제거 후 숫자만 사용
        (r'\(01\)', 'REMOVE', 'GS1 AI (01)'),
        (r'\(10\)', 'REMOVE', 'GS1 AI (10)'),
        (r'\(11\)', 'REMOVE', 'GS1 AI (11)'),
        (r'\(17\)', 'REMOVE', 'GS1 AI (17)'),
        (r'\(21\)', 'REMOVE', 'GS1 AI (21)'),
        
        (r'[\-_]+', 'REMOVE', '하이픈/언더스코어'),
        
        (r'\(\d{2}\)', 'REMOVE', 'GS1 AI 일반'),
        
        # HIBCC/ICCBBA 접두어 - 제거 가능
        (r'^\d*\+', 'REMOVE', '숫자+HIBCC 접두어'),
        
        # 이상한 특수문자 - 제거 후 재평가
        (r'[\$\@\#\&\*]+', 'REMOVE', '특수문자 ($@#&*)'),
        (r'[\\\/]+', 'REMOVE', '슬래시'),
        
        # 텍스트 래퍼 제거
        (r'^[\[\]\{\}\<\>]+', 'REMOVE', '괄호 래퍼 앞'),
        (r'[\[\]\{\}\<\>]+$', 'REMOVE', '괄호 래퍼 뒤'),
        (r'^["\']', 'REMOVE', '따옴표 앞'),
        (r'["\']$', 'REMOVE', '따옴표 뒤'),
    ]
    
    # 추가 검증 규칙
    ADDITIONAL_CHECKS = [
        # 함수 기반 검증 (정규표현식으로 표현하기 어려운 경우)
        ('check_all_same_char', 'DELETE', '모두 같은 문자'),
        ('check_invalid_chars', 'DELETE', '허용되지 않는 문자'),
        ('check_ratio_invalid', 'DELETE', '유효하지 않은 문자 비율'),
    ]
    
    @staticmethod
    def check_all_same_char(udi: str) -> bool:
        """모든 문자가 동일한지 체크 (길이 6 이상)"""
        if len(udi) >= 6:
            return len(set(udi)) == 1
        return False
    
    @staticmethod
    def check_invalid_chars(udi: str) -> bool:
        """허용되지 않는 문자 포함 여부"""
        # 허용: 숫자, 알파벳, +, =, -, /, (, ), :, .
        allowed_pattern = r'^[0-9A-Za-z+=\-/():.]+$'
        return not re.match(allowed_pattern, udi)
    
    @staticmethod
    def check_ratio_invalid(udi: str, threshold: float = 0.3) -> bool:
        """유효하지 않은 문자의 비율이 임계값 이상인지"""
        if not udi:
            return True
        
        invalid_chars = len([c for c in udi if not c.isalnum() and c not in '+=-./:()'])
        ratio = invalid_chars / len(udi)
        return ratio > threshold


def process_invalid_patterns(udi: str, verbose: bool = False) -> Tuple[Optional[str], str, list]:
    """
    UDI에 대해 이상 패턴 처리를 수행
    
    Args:
        udi: 처리할 UDI 문자열
        verbose: 상세 로그 출력 여부
        
    Returns:
        (processed_udi, action, applied_patterns)
        - processed_udi: 처리된 UDI (None이면 삭제)
        - action: 'DELETE', 'CLEANED', 'VALID'
        - applied_patterns: 적용된 패턴 목록
    """
    if not udi:
        return None, 'DELETE', [('empty', 'DELETE')]
    
    original = udi
    applied_patterns = []
    was_modified = False
    
    # Step 1: REMOVE 패턴 적용 (패턴만 제거)
    for pattern, action, desc in ImprovedInvalidPatterns.INVALID_PATTERNS:
        if action == 'REMOVE':
            if re.search(pattern, udi, re.IGNORECASE):
                before = udi
                udi = re.sub(pattern, '', udi, flags=re.IGNORECASE)
                if before != udi:
                    applied_patterns.append((desc, 'REMOVE'))
                    was_modified = True
                    if verbose:
                        print(f"  REMOVE [{desc}]: '{before}' → '{udi}'")
    
    # Step 2: REMOVE 후 남은 문자열 정리
    udi = udi.strip()
    
    # Step 3: DELETE 패턴 체크 (완전 삭제)
    for pattern, action, desc in ImprovedInvalidPatterns.INVALID_PATTERNS:
        if action == 'DELETE':
            if re.match(pattern, udi, re.IGNORECASE):
                applied_patterns.append((desc, 'DELETE'))
                if verbose:
                    print(f"  DELETE [{desc}]: '{udi}' → (삭제)")
                return None, 'DELETE', applied_patterns
    
    # Step 4: 추가 함수 기반 검증
    for check_name, action, desc in ImprovedInvalidPatterns.ADDITIONAL_CHECKS:
        check_func = getattr(ImprovedInvalidPatterns, check_name)
        if check_func(udi):
            applied_patterns.append((desc, action))
            if verbose:
                print(f"  {action} [{desc}]: '{udi}'")
            if action == 'DELETE':
                return None, 'DELETE', applied_patterns
    
    # Step 5: 최종 결과 반환
    if not udi or len(udi) == 0:
        return None, 'DELETE', applied_patterns + [('empty after cleaning', 'DELETE')]
    
    if was_modified:
        return udi, 'CLEANED', applied_patterns
    else:
        return udi, 'VALID', []


def demo_pattern_processing():
    """이상 패턴 처리 데모"""
    test_cases = [
        # DELETE 케이스
        'UNKNOWN',
        '00000000',
        'N/A',
        'TEST123',
        '11111111',
        'X',
        
        # REMOVE 케이스 (수정 가능)
        '00(01)(21)5121',
        '0+M724CCB541',
        '  12345678  ',
        '123-456-789',
        '$$ABC123$$',
        '"12345678"',
        
        # VALID 케이스
        '00012345678901',
        'M2138071M15K15',
        '+M724CCB541',
    ]
    
    print("="*80)
    print("이상 패턴 처리 데모")
    print("="*80)
    print(f"{'원본':<25} {'처리 결과':<25} {'액션':<10} {'적용 패턴'}")
    print("-"*80)
    
    for udi in test_cases:
        processed, action, patterns = process_invalid_patterns(udi, verbose=False)
        result = processed if processed else '(삭제)'
        pattern_desc = ', '.join([p[0] for p in patterns[:2]]) if patterns else '-'
        if len(patterns) > 2:
            pattern_desc += f' 외 {len(patterns)-2}개'
        print(f"{udi:<25} {result:<25} {action:<10} {pattern_desc}")
    
    print("="*80)


def show_pattern_categories():
    """패턴 카테고리별 출력"""
    print("\n" + "="*80)
    print("이상 패턴 카테고리")
    print("="*80)
    
    delete_patterns = [(p, d) for p, a, d in ImprovedInvalidPatterns.INVALID_PATTERNS if a == 'DELETE']
    remove_patterns = [(p, d) for p, a, d in ImprovedInvalidPatterns.INVALID_PATTERNS if a == 'REMOVE']
    
    print(f"\n📛 DELETE 패턴 (완전 삭제): {len(delete_patterns)}개")
    print("-"*80)
    for i, (pattern, desc) in enumerate(delete_patterns, 1):
        print(f"{i:2d}. {desc:<30} | {pattern}")
    
    print(f"\n✂️  REMOVE 패턴 (패턴만 제거): {len(remove_patterns)}개")
    print("-"*80)
    for i, (pattern, desc) in enumerate(remove_patterns, 1):
        print(f"{i:2d}. {desc:<30} | {pattern}")
    
    print(f"\n🔍 추가 검증 규칙: {len(ImprovedInvalidPatterns.ADDITIONAL_CHECKS)}개")
    print("-"*80)
    for i, (func, action, desc) in enumerate(ImprovedInvalidPatterns.ADDITIONAL_CHECKS, 1):
        print(f"{i:2d}. {desc:<30} | {func}() [{action}]")
    
    print("="*80)


if __name__ == "__main__":
    # 패턴 카테고리 출력
    show_pattern_categories()
    
    # 데모 실행
    demo_pattern_processing()
    
    # 상세 예제
    print("\n" + "="*80)
    print("상세 처리 예제")
    print("="*80)
    
    examples = [
        '00(01)(21)5121',
        '  +M724CCB541  ',
        '$$TEST-123$$',
    ]
    
    for udi in examples:
        print(f"\n원본: '{udi}'")
        processed, action, patterns = process_invalid_patterns(udi, verbose=True)
        print(f"결과: '{processed}' [{action}]")


이상 패턴 카테고리

📛 DELETE 패턴 (완전 삭제): 21개
--------------------------------------------------------------------------------
 1. 모두 0                           | ^0+$
 2. 8개 이상 연속된 0                    | ^0{8,}$
 3. 같은 숫자만 반복 (11111, 22222 등)     | ^(\d)\1+$
 4. 빈 문자열                          | ^\s*$
 5. 단일 문자 N                        | ^N$
 6. X만 반복                          | ^X+$
 7. UNKNOWN                        | ^UNKNOWN$
 8. UNK/UNKNOWN                    | ^UNK(NOWN)?$
 9. N/A 또는 NA                      | ^N/?A$
10. NULL                           | ^NULL$
11. NONE                           | ^NONE$
12. NOT AVAILABLE                  | ^NOT[\s_]?AVAILABLE$
13. UNAVAILABLE                    | ^UNAVAILABLE$
14. MISSING                        | ^MISSING$
15. NO DATA                        | ^NO[\s_]?DATA$
16. EMPTY                          | ^EMPTY$
17. TEST 데이터                       | ^TEST\d*$
18. DUMMY 데이터                      | ^DUMMY$
19. SAMPLE 데이터                     | ^SAMPLE$
2

In [ ]:
"""
UDI-DI 이상 패턴 처리 모듈
패턴을 유연하게 처리하며, 제거/삭제 액션을 분리
"""

import re
from typing import Tuple, Optional


class InvalidPatternHandler:
    """이상 패턴 탐지 및 처리 클래스"""
    
    # 패턴 정의: (정규표현식, 액션, 설명)
    # 액션: 'DELETE' = 전체 삭제 (NULL 반환), 'REMOVE' = 패턴 부분만 제거
    PATTERNS = [
        # ===== DELETE 패턴 (전체 삭제) =====
        (r'^0+$', 'DELETE', '모두 0'),
        (r'^0{8,}$', 'DELETE', '8개 이상 연속된 0'),
        (r'^\s*$', 'DELETE', '빈 문자열'),
        (r'^N$', 'DELETE', '단일 문자 N'),
        
        # UNKNOWN 관련 (다양한 변형 포함)
        (r'.*\bUNKNOWN\b.*', 'DELETE', 'UNKNOWN 포함'),
        (r'.*\bUNK\b.*', 'DELETE', 'UNK 포함'),
        (r'.*\bUNKOWN\b.*', 'DELETE', 'UNKOWN (오타) 포함'),
        (r'.*\bUKN\b.*', 'DELETE', 'UKN (오타) 포함'),
        
        # NULL 관련
        (r'.*\bNULL\b.*', 'DELETE', 'NULL 포함'),
        (r'.*\bNONE\b.*', 'DELETE', 'NONE 포함'),
        (r'.*\bNIL\b.*', 'DELETE', 'NIL 포함'),
        
        # N/A 관련 (다양한 표기)
        (r'.*\bN/?A\b.*', 'DELETE', 'N/A 또는 NA 포함'),
        (r'.*\bNOT\s+AVAILABLE\b.*', 'DELETE', 'NOT AVAILABLE 포함'),
        (r'.*\bUNAVAILABLE\b.*', 'DELETE', 'UNAVAILABLE 포함'),
        (r'.*\bN\.A\.?\b.*', 'DELETE', 'N.A 포함'),
        
        # 특수 케이스
        (r'.*\(01\).*\bNA\b.*', 'DELETE', '(01)과 NA 조합'),
        (r'.*\(01\)\(10\)\bNA\b.*', 'DELETE', '(01)(10)NA 조합'),
        
        # 이상 특수문자
        (r'.*\$\$.*', 'DELETE', '$$ 포함'),
        (r'.*[@#%^&*]{2,}.*', 'DELETE', '연속 특수문자'),
        
        # 모두 같은 숫자/문자
        (r'^(.)\1{7,}$', 'DELETE', '8개 이상 같은 문자'),
        
        # ===== REMOVE 패턴 (부분 제거) =====
        # 괄호로 묶인 Application Identifier는 제거만
        (r'\([^)]*\)', 'REMOVE', '괄호 안 내용 제거'),
        
        # 앞쪽의 0과 +, = 조합 (HIBCC/ICCBBA 처리용)
        (r'^0+(?=[+=])', 'REMOVE', '앞쪽 0 (기호 앞)'),
        
        # 공백 제거
        (r'\s+', 'REMOVE', '공백 제거'),
        
        # 콤마, 세미콜론 제거
        (r'[,;]', 'REMOVE', '구분자 제거'),
    ]
    
    def __init__(self):
        """초기화 및 패턴 컴파일"""
        self.compiled_patterns = []
        for pattern, action, desc in self.PATTERNS:
            self.compiled_patterns.append({
                'regex': re.compile(pattern, re.IGNORECASE),
                'action': action,
                'description': desc,
                'original': pattern
            })
        
        self.stats = {
            'total_checked': 0,
            'deleted': 0,
            'removed': 0,
            'kept': 0,
            'patterns_matched': {}
        }
    
    def process(self, udi: str) -> Tuple[Optional[str], str, str]:
        """
        UDI를 이상 패턴에 따라 처리
        
        Args:
            udi: 입력 UDI 문자열
            
        Returns:
            (processed_udi, status, matched_pattern_desc)
            - processed_udi: 처리된 UDI (None이면 삭제)
            - status: 'DELETED', 'REMOVED', 'KEPT'
            - matched_pattern_desc: 매칭된 패턴 설명
        """
        self.stats['total_checked'] += 1
        
        if not udi or len(udi) == 0:
            self.stats['deleted'] += 1
            return None, 'DELETED', '빈 문자열'
        
        original_udi = udi
        
        # 패턴 순회
        for pattern_info in self.compiled_patterns:
            regex = pattern_info['regex']
            action = pattern_info['action']
            desc = pattern_info['description']
            
            if regex.match(udi):
                # 통계 업데이트
                if desc not in self.stats['patterns_matched']:
                    self.stats['patterns_matched'][desc] = 0
                self.stats['patterns_matched'][desc] += 1
                
                if action == 'DELETE':
                    # 전체 삭제
                    self.stats['deleted'] += 1
                    return None, 'DELETED', desc
                
                elif action == 'REMOVE':
                    # 패턴 부분만 제거
                    udi = regex.sub('', udi).strip()
                    
                    # 제거 후 빈 문자열이 되면 삭제
                    if not udi or len(udi) == 0:
                        self.stats['deleted'] += 1
                        return None, 'DELETED', f'{desc} (제거 후 빈 문자열)'
                    
                    # 제거 후에도 유효하면 계속 진행
                    continue
        
        # 모든 REMOVE 패턴 적용 후 결과 확인
        if udi != original_udi:
            # 패턴이 제거됨
            self.stats['removed'] += 1
            return udi, 'REMOVED', '패턴 제거됨'
        
        # 패턴에 걸리지 않음
        self.stats['kept'] += 1
        return udi, 'KEPT', '정상'
    
    def process_batch(self, udi_list: list) -> dict:
        """
        배치 처리
        
        Returns:
            {
                'deleted': [...],
                'removed': [(original, processed), ...],
                'kept': [...]
            }
        """
        result = {
            'deleted': [],
            'removed': [],
            'kept': []
        }
        
        for udi in udi_list:
            processed, status, desc = self.process(udi)
            
            if status == 'DELETED':
                result['deleted'].append({
                    'original': udi,
                    'reason': desc
                })
            elif status == 'REMOVED':
                result['removed'].append({
                    'original': udi,
                    'processed': processed,
                    'reason': desc
                })
            else:
                result['kept'].append(udi)
        
        return result
    
    def add_custom_pattern(self, pattern: str, action: str, description: str):
        """
        커스텀 패턴 추가
        
        Args:
            pattern: 정규표현식 패턴
            action: 'DELETE' 또는 'REMOVE'
            description: 패턴 설명
        """
        if action not in ['DELETE', 'REMOVE']:
            raise ValueError("action은 'DELETE' 또는 'REMOVE'여야 합니다.")
        
        self.compiled_patterns.append({
            'regex': re.compile(pattern, re.IGNORECASE),
            'action': action,
            'description': description,
            'original': pattern
        })
    
    def print_stats(self):
        """통계 출력"""
        print("\n" + "="*70)
        print("이상 패턴 처리 통계")
        print("="*70)
        print(f"전체 확인:         {self.stats['total_checked']:,}")
        print(f"  삭제 (DELETE):   {self.stats['deleted']:,} ({self.stats['deleted']/max(1, self.stats['total_checked'])*100:.1f}%)")
        print(f"  부분 제거:       {self.stats['removed']:,} ({self.stats['removed']/max(1, self.stats['total_checked'])*100:.1f}%)")
        print(f"  정상 유지:       {self.stats['kept']:,} ({self.stats['kept']/max(1, self.stats['total_checked'])*100:.1f}%)")
        print("="*70)
        
        if self.stats['patterns_matched']:
            print("\n매칭된 패턴 TOP 10:")
            sorted_patterns = sorted(
                self.stats['patterns_matched'].items(),
                key=lambda x: x[1],
                reverse=True
            )
            for desc, count in sorted_patterns[:10]:
                print(f"  {desc:<40} {count:>6}회")
    
    def get_pattern_list(self):
        """등록된 패턴 목록 반환"""
        patterns = []
        for p in self.compiled_patterns:
            patterns.append({
                'pattern': p['original'],
                'action': p['action'],
                'description': p['description']
            })
        return patterns


# ===== 사용 예시 =====
if __name__ == "__main__":
    handler = InvalidPatternHandler()
    
    print("="*70)
    print("UDI-DI 이상 패턴 처리 테스트")
    print("="*70)
    
    # 테스트 케이스
    test_cases = [
        # DELETE 케이스
        'UNKNOWN',
        '(01)UNKNOWN',
        'UNK123',
        '00000000',
        'NULL',
        'N/A',
        'NOT AVAILABLE',
        '(01)(10)NA',
        'N',
        '11111111',
        '$$ERROR$$',
        
        # REMOVE 케이스
        '00(01)(21)5121',
        '0+M724CCB541',
        '  12345678  ',
        '12,345,678',
        '(01)12345678',
        
        # KEPT 케이스
        '00012345678901',
        '+M724CCB541',
        'M2138071M15K15',
    ]
    
    print("\n개별 처리 테스트:")
    print(f"{'원본':<30} {'처리 결과':<30} {'상태':<12} {'사유'}")
    print("-"*70)
    
    for udi in test_cases:
        processed, status, reason = handler.process(udi)
        result = processed if processed else '(삭제됨)'
        print(f"{udi:<30} {result:<30} {status:<12} {reason}")
    
    handler.print_stats()
    
    # 배치 처리 테스트
    print("\n" + "="*70)
    print("배치 처리 테스트")
    print("="*70)
    
    batch_result = handler.process_batch(test_cases)
    
    print(f"\n삭제된 UDI: {len(batch_result['deleted'])}개")
    for item in batch_result['deleted'][:5]:
        print(f"  {item['original']:<30} → {item['reason']}")
    
    print(f"\n부분 제거된 UDI: {len(batch_result['removed'])}개")
    for item in batch_result['removed'][:5]:
        print(f"  {item['original']:<30} → {item['processed']:<30} ({item['reason']})")
    
    print(f"\n정상 유지된 UDI: {len(batch_result['kept'])}개")
    for udi in batch_result['kept'][:5]:
        print(f"  {udi}")
    
    # 커스텀 패턴 추가 예시
    print("\n" + "="*70)
    print("커스텀 패턴 추가 예시")
    print("="*70)
    
    handler.add_custom_pattern(r'TEST\d+', 'DELETE', '테스트 데이터')
    
    test_custom = ['TEST123', 'TEST456', 'NORMAL123']
    for udi in test_custom:
        processed, status, reason = handler.process(udi)
        result = processed if processed else '(삭제됨)'
        print(f"{udi:<20} → {result:<20} ({status}: {reason})")
    
    # 등록된 패턴 목록 출력
    print("\n" + "="*70)
    print("등록된 전체 패턴 목록")
    print("="*70)
    patterns = handler.get_pattern_list()
    for i, p in enumerate(patterns, 1):
        print(f"{i:2}. [{p['action']:<6}] {p['description']:<30} | {p['pattern']}")